## Getting the Data
To get the data run the KNMI_open_data_API.py script (in case of error check API key expiry date)

This downloads 10 tar files, these are the most recent iterations of prediction computations, we are only interested in the last downloaded file. One file is computed every 6 hours giving new predictions for the following 48 hours with naming convention harm40_v1_P1_jjjjmmddhh.tar. Computation takes approx 3 hours, so these are timestamped for start of computation not completion!

it contains predictions for the upcoming 48 hours of weather data. For each timestep there is one grib file so in total 49 timesteps

To extract the tar file:

In [ ]:
import tarfile
tf = tarfile.open('./dataset-download/harm40_v1_p1_2021070606.tar', mode='r')
tf.extractall('./dataset-download/')

To handle the returned file take a look at the table in: https://www.knmidata.nl/data-services/knmi-producten-overzicht/atmosfeer-modeldata/data-product-1

One oddity of pygrib is that once you have iterated through the list you have to reopen it

To open one of the extracted files with naming convention HA40_N25_yyyymmddhhmm_00t00 where t is the timestep in hours:

In [55]:
import pygrib
#The Below file then corresponds to 06-07-2021 - 6am with a 6 hour timestep taken
gr = pygrib.open('./dataset-download/HA40_N25_202107060600_00600_GB')

## Data Extraction

From the above mentioned table select which variables are required, for the purposes of this illustration we will be using "Totale hoeveelheid bewolking" (TCC) with code=71, leveltype=105, level=0, tri=0, in units of %


In [52]:
(code, leveltype, level, tri) = (71,105,0,0)
for g in gr: #https://jswhit.github.io/pygrib/api.html#example-usage #TODO: Figure this out
    print(g)

In [44]:
print(gr[24])
g = gr[24]
print({key:(g[key] if g.valid_key(key) else 'err') for key in g.keys()})

24:Temperature:K (instant):regular_ll:heightAboveGround:level 0:fcst time 6 hrs:from 202107060600
{'globalDomain': 'g', 'GRIBEditionNumber': 1, 'eps': 0, 'offsetSection0': 0, 'section0Length': 8, 'totalLength': 270612, 'editionNumber': 1, 'WMO': 0, 'productionStatusOfProcessedData': 0, 'section1Length': 28, 'wrongPadding': 0, 'table2Version': 1, 'centre': '99', 'centreDescription': 'DeBilt, Netherlands', 'generatingProcessIdentifier': 202, 'gridDefinition': 255, 'indicatorOfParameter': 11, 'parameterName': 'T Temperature K', 'parameterUnits': 'unknown', 'indicatorOfTypeOfLevel': 'sfc', 'pressureUnits': 'hPa', 'typeOfLevelECMF': 'heightAboveGround', 'typeOfLevel': 'heightAboveGround', 'level': 0, 'yearOfCentury': 21, 'month': 7, 'day': 6, 'hour': 6, 'minute': 0, 'second': 0, 'unitOfTimeRange': 1, 'P1': 6, 'P2': 0, 'timeRangeIndicator': 0, 'numberIncludedInAverage': 0, 'numberMissingFromAveragesOrAccumulations': 0, 'centuryOfReferenceTimeOfData': 21, 'subCentre': 0, 'paramIdECMF': '130',

In [61]:
import os 
root = "/home/felix/ldst-control-systems/"
path = os.path.join(root, "dataset-download")
gr = pygrib.open('./dataset-download/HA40_N25_202107060600_00600_GB')
for path, subdirs, files in os.walk(path):
    for name in files:
        if name[-5::]=='00_GB':
            gr = pygrib.open('./dataset-download/{}'.format(name))
            print('-'*50)
            print(name)
            print('-'*50)
            for g in gr:
                if g.name !='unknown':
                    print(g)
                    print('\n')
                    print(g.name)
                    print('-------------')
                    print(g.indicatorOfParameter)
                    print('-------------')
                    print(g.parameterName)
                    print('-------------')
                    print(g.cfVarName)
                    print('\n')
                    print({key:(g[key] if g.valid_key(key) else 'err') for key in g.keys()})
                    print('\n')
                    print('\n')

--------------------------------------------------
HA40_N25_202107060600_04700_GB
--------------------------------------------------
23:Snow depth:m (instant):regular_ll:heightAboveGround:level 0:fcst time 47 hrs:from 202107060600


Snow depth
-------------
66
-------------
SD Snow depth m
-------------
sd


{'globalDomain': 'g', 'GRIBEditionNumber': 1, 'eps': 0, 'offsetSection0': 0, 'section0Length': 8, 'totalLength': 175074, 'editionNumber': 1, 'WMO': 0, 'productionStatusOfProcessedData': 0, 'section1Length': 28, 'wrongPadding': 0, 'table2Version': 1, 'centre': '99', 'centreDescription': 'DeBilt, Netherlands', 'generatingProcessIdentifier': 202, 'gridDefinition': 255, 'indicatorOfParameter': 66, 'parameterName': 'SD Snow depth m', 'parameterUnits': 'of water equivalent', 'indicatorOfTypeOfLevel': 'sfc', 'pressureUnits': 'hPa', 'typeOfLevelECMF': 'heightAboveGround', 'typeOfLevel': 'heightAboveGround', 'level': 0, 'yearOfCentury': 21, 'month': 7, 'day': 6, 'hour': 6, 'minute': 0, 'sec